# Get Training Data

In [30]:
import os
import sys
from pathlib import Path # if you haven't already done so
file = Path(os.getcwd()).resolve()
parent, root = file.parent, file.parents[1]
print(parent, root)
sys.path.append(str(parent))
# sys.path.append("..")

# # Additionally remove the current file's directory from sys.path
# try:
#     sys.path.remove(str(parent))
# except ValueError: # Already removed
#     pass

/home/VD/fruitao/drone/logic_guided_prompt/code /home/VD/fruitao/drone/logic_guided_prompt


In [31]:
from logic.description_logic import *
from utils.load_sensor_data import *
from data_augmentation.text_augmentation import *

In [32]:
# old_format_example=pd.ExcelFile(r"../text_annotation/rural.xlsx")

dataset_file_li=[r"../../text_annotation/desert.xlsx", r"../../text_annotation/disturbance.xlsx", r"../../text_annotation/factory.xlsx", r"../../text_annotation/island.xlsx", 
                 r"../../text_annotation/misc.xlsx", r"../../text_annotation/ocean.xlsx", r"../../text_annotation/rural.xlsx", r"../../text_annotation/urban.xlsx"
]

old_data_dic=dict()

for dataset_file in dataset_file_li:
    print("----processing file: {}----".format(dataset_file))
    
    old_format_example=pd.ExcelFile(dataset_file)

    ts_converters={'name': str, 'Type': str, 'Moving': str2bool, 'InPath': str2bool, 'time_stamp': str}
    status_converters={'PilotExperienced': str2bool, 'Low_visibility': str2bool, 'Normal_frame': str2bool,
                      'weather': str, 'upside_down': str2bool, 'good_motor_condition': str2bool,
                      'going_backwards': str2bool, 'indoor': str2bool, 'waterproof drone?': str2bool,
                      'flying over': str, 'criticality': str, 
                      'RiskOfPhysicalDamage': str, 'RiskOfInternalDamage': str2bool,
                      'RiskOfHumanDamage': str2bool, 'LostConnection': str2bool}
    text_converters={'Link':str, 'Text1':str, 'Text2':str}

    old_format_ts=pd.read_excel(old_format_example, sheet_name='timestep', converters=ts_converters)
    old_format_status=pd.read_excel(old_format_example, sheet_name='status', converters=status_converters)
    old_format_text=pd.read_excel(old_format_example, sheet_name='text', converters=text_converters)

    print("---loading sensor data---")
    load_ts_to_dict(old_format_ts, old_data_dic)
    load_status_to_dict(old_format_status, old_data_dic)
    load_text_to_dict(old_format_text, old_data_dic)

    # print("---generating templates---")
    for link, data in old_data_dic.items():
        templates, related_status_dic, related_timestep_dic = gen_templates(data)
        old_data_dic[link]['templates'] = templates
        old_data_dic[link]['related_status_dic'] = related_status_dic
        old_data_dic[link]['related_timestep_dic'] = related_timestep_dic


----processing file: ../../text_annotation/desert.xlsx----
---loading sensor data---
--gen_templates-- related_status_dic:{'flying_not_normal_frame': [{'Normal_frame': False}]}
--gen_templates-- template:The drone is flying with a damaged frame.
templates:{'flying_not_normal_frame': ['The drone is flying with a damaged frame.'], 'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:05) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.3m."], 'reachable_inpath': ["(0:05) The drone is facing the risk of physical damage. There is a car in the drone's flight path at a distance of 0.3m."]}
templates:{'near_moving_object': ["(0:01) The drone is risking physical damage. It's flying too close to the moving car at a distance of 3.0m.", "(0:03) The drone is risking physical damage. It's flying too close to the moving car at a distance of 0.2m."], 'reachable_inpath': 

In [33]:
# print("old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['timestep'][0]['time_stamp']:{}".format(old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['timestep'][0]['time_stamp']))
print("len(old_data_dic):{}".format(len(old_data_dic)))
print("--['text'][0]:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['text'][0]))
print("--['related_status_dic']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['related_status_dic']))
print("--['related_timestep_dic']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['related_timestep_dic']))
print("--['templates']:{}".format(
    old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing']['templates']))

len(old_data_dic):300
--['text'][0]:{'Text1': "Risk of Physical Damage! The drone has a damaged frame and empty battery. It's flying at an altitude of 5m. [SEP]\n(0:01) Risk of Physical Damage! There are two gravestones in the drone's flight path at a distance of 3m.  [SEP] \n(0:05) Risk of Physical Damage! Now there's a castle in the drone's flight path at a distance of 1m. [SEP]", 'Text2': "Risk of physical damage! The drone is flying with an empty battery. The frame is also damaged and the altitude is 5m. [SEP]\n(0:01) Risk of physical damage! There are two gravestones 3m away and in the drone's path. [SEP]\n(0:05) Risk of physical damage! The drone is flying toward a castle in its path which is only 1m away. [SEP]"}
--['related_status_dic']:{'flying_empty_battery': [{'Battery_level': 0}], 'flying_not_normal_frame': [{'Normal_frame': False}]}
--['related_timestep_dic']:{'reachable_inpath': [{'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name

In [ ]:
print(old_data_dic['https://drive.google.com/file/d/1oJ6t-VJN3rAmslsXszkVUzTZl4LJujeM/view?usp=sharing'].keys())

In [ ]:
import json

link_li=[]
text_li=[]
dl_name_li=[]
templates_li=[]
related_sensor_data_li=[]
related_status_dic_li=[]
related_timestep_dic_li=[]
dl_related_sensor_data_mapping_li=[]

# related_sensor_data_li as input and templates_li as label
for link, value in old_data_dic.items():
#     print("value:{}".format(value))
    link_li.append(link)
    if 'text' not in value:
        text_li.append([])
    else:    
        text_li.append(value['text'])
    dl_index=0
    
    for dl_name, template_value in value['templates'].items():
        templates_li.extend(template_value)
        if dl_name in value['related_status_dic']:
            related_status_dic_li.extend(value['related_status_dic'][dl_name])
            related_sensor_data_li.extend(value['related_status_dic'][dl_name])
        elif dl_name in value['related_timestep_dic']:
            related_timestep_dic_li.extend(value['related_timestep_dic'][dl_name])
            related_sensor_data_li.extend(value['related_timestep_dic'][dl_name])
        else:
            print("WHAT?")
        
related_sensor_data_li=[json.dumps(dic) for dic in related_sensor_data_li]

In [ ]:
import math

def train_test_split(all_li, train_ratio=0.8， val_ratio=0.1):
    all_len=len(all_li)
    train_len=math.ceil(train_ratio*all_len)
    val_len=math.ceil(val_ratio*all_len)
    
    return all_li[:train_len], all_li[train_len:train_len+val_len], all_li[all_li[train_len:train_len+val_len:]

train_sensor_data, val_sensor_data, test_sensor_data=train_test_split(related_sensor_data_li)
train_templates, val_templates, test_templates=train_test_split(templates_li)

print(len(related_sensor_data_li), len(train_sensor_data), len(test_sensor_data))
print(len(templates_li), len(train_templates), len(test_templates))
    

In [ ]:
# mapping{'flying_empty_battery':[0], 'flying_not_normal_frame':[1], 'reachable_inpath':[2, 3, 4]}
# input: [{'Battery_level': 0}, {'Normal_frame': False}, {'time_stamp': '0:01', 'name': 'gravestone 1', 'Distance': 3.0}, {'time_stamp': '0:01', 'name': 'gravestone 2', 'Distance': 3.0}, {'time_stamp': '0:05', 'name': 'castle ', 'Distance': 1.0}]
# output: 'The flying drone is runing out of battery with only 0% charge.'

print(len(related_sensor_data_li), related_sensor_data_li[0], type(related_sensor_data_li[0]))
print(len(templates_li), templates_li[0], type(templates_li[0]))
# print(dl_related_sensor_data_mapping_li[2])

# Augmented Prompt

In [ ]:
!nvidia-smi

In [ ]:
import torch

device=torch.device("cuda:1")

In [ ]:
import math

from tqdm import tqdm
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch.nn as nn
from sklearn.metrics import accuracy_score
from nltk.translate import meteor_score

In [ ]:
Torch.manual_seed(42)

In [ ]:
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                               n_tokens, 
                                                                               random_range, 
                                                                               initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
n_tokens = len(WARNING_DL)
initialize_from_vocab = True
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.to(device)

In [ ]:
parameters = list(model.parameters())

for x in parameters[1:]:
    x.requires_grad = False

In [ ]:
str_warning_dl_list=[dl_func.__name__ for dl_func in WARNING_DL]
print(str_warning_dl_list)

In [ ]:
# Todo: Use dataset instead.

def generate_data(batch_size, n_tokens, related_sensor_data, text_data):

    def yield_data(x_batch, y_batch, l_batch):
            
        x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
        y = torch.nn.utils.rnn.pad_sequence(y_batch, batch_first=True)
#         y = torch.cat(y_batch, dim=0)
            
        # Todo: Add prompt mask for different description logic
        m = (x > 0).to(torch.float32)
        decoder_input_ids = torch.full((x.shape[0], y.shape[1]), 1)
        
        if torch.cuda.is_available():
            x = x.to(device)
            y = y.to(device)
            m = m.to(device)
            decoder_input_ids = decoder_input_ids.to(device)
        return x, y, m, decoder_input_ids, l_batch

    x_batch, y_batch, l_batch = [], [], []
    for x, y in zip(related_sensor_data, text_data):
        
        context = x
        inputs = tokenizer(context, return_tensors="pt")
        inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        
        outputs=tokenizer(y, return_tensors="pt")
#         print(type(inputs), inputs['input_ids'][0].size(), inputs['input_ids'].size(), 
#               type(outputs), outputs['input_ids'][0].size(), outputs['input_ids'].size())
        
        outputs_ids = torch.cat([torch.full((1, n_tokens - 1), -100), outputs['input_ids']], 1)
        
        x_batch.append(inputs['input_ids'][0])
        y_batch.append(outputs_ids[0])
        l_batch.append(outputs['input_ids'][0])
        if len(x_batch) >= batch_size:
            yield yield_data(x_batch, y_batch, l_batch)
            x_batch, y_batch, l_batch = [], [], []

    if len(x_batch) > 0:
        yield yield_data(x_batch, y_batch, l_batch)
        x_batch, y_batch, l_batch = [], [], []

In [ ]:
# Test generation function
for x, y, m, dii, true_labels in generate_data(4, n_tokens, related_sensor_data_li, templates_li):
    assert dii.shape == y.shape
    outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
    assert outputs['logits'].shape[:2] == y.shape
    pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy()
    print("pred_labels:{}".format((pred_labels, pred_labels.shape)))
    break

In [ ]:
batch_size = 32
n_epoch = 300
# total_batch = math.ceil(len(related_sensor_data_li) / batch_size)
# dev_total_batch = math.ceil(len(templates_li) / batch_size)
total_batch = math.ceil(len(train_sensor_data) / batch_size)
dev_total_batch = math.ceil(len(test_sensor_data) / batch_size)
use_ce_loss = False
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

test_losses_epoch=[]
bleu_score_epoch=[]
rouge_score_epoch=[]
meteor_score_epoch=[]

from datasets import load_metric

bleu=load_metric("bleu")
rouge=load_metric("rouge")
meteor = load_metric('meteor')

for epoch in range(n_epoch):
    print('epoch', epoch)

    all_true_labels = []
    all_pred_labels = []
    train_losses = []
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, train_sensor_data, train_templates)), total=total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        
        optimizer.zero_grad()
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels

        if use_ce_loss:
            logits = outputs['logits'][:, :, :]
            true_labels_tensor = torch.tensor(true_labels, dtype=torch.long).cuda()
            loss = ce_loss(logits, true_labels_tensor)
        else:
            loss = outputs.loss
        loss.backward()
        optimizer.step()
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        train_losses.append(loss_value)

#         acc = accuracy_score(all_true_labels, all_pred_labels)
        pbar.set_description(f'train: loss={np.mean(train_losses):.4f}')

    all_true_labels = []
    all_pred_labels = []
    output_max_pred=[]
    test_losses = []
    
    with torch.no_grad():
        pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, test_sensor_data, test_templates)), total=dev_total_batch)
        for i, (x, y, m, dii, true_labels) in pbar:
            all_true_labels += true_labels
            outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
            
            output_ids=outputs.logits[:, n_tokens:, :]
            max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
            max_pred_ids[0]=[ids for ids in max_pred_ids[0] if ids!=0]
            output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
            
            loss = outputs.loss
            loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
            test_losses.append(loss_value)
            
            pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
            all_pred_labels += pred_labels
            pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')
            
    bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
    bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]
    
    meteor_output = [meteor_score.single_meteor_score(
        tokenizer.tokenize(ref_s), tokenizer.tokenize(pred_s), alpha=0.9, beta=3, gamma=0.5)
                     for ref_s, pred_s in zip(output_max_pred, test_templates)
                    ]
    meteor_avg_score=sum(meteor_output) / len(meteor_output)
    
    bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
    rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)
    
    bleu_score_epoch.append(bleu_score)
    rouge_score_epoch.append(rouge_score)
    meteor_score_epoch.append(meteor_avg_score)
    
    test_losses_epoch.append(np.mean(test_losses))

epoch 2


dev: loss=0.1265: 100%|███████████████████████████████████████| 21/21 [00:03<00:00,  5.82it/s]


epoch 3


dev: loss=0.1093: 100%|███████████████████████████████████████| 21/21 [00:03<00:00,  5.61it/s]


epoch 4


...

dev: loss=0.0536: 100%|███████████████████████████████████████| 21/21 [00:03<00:00,  6.07it/s]


epoch 293


dev: loss=0.0543: 100%|███████████████████████████████████████| 21/21 [00:03<00:00,  6.03it/s]


epoch 294


dev: loss=0.0555: 100%|███████████████████████████████████████| 21/21 [00:03<00:00,  6.00it/s]


In [ ]:
print("test_sensor_data: ", len(test_sensor_data))
# print("all_pred_labels:", all_pred_labels, len(all_pred_labels), len(all_pred_labels[0]))
print("output_max_pred:", output_max_pred[0], len(output_max_pred))
# print(len(test_losses), len(test_sensor_data)/32, len(test_losses_epoch[0]))

# print("bleu_score_epoch", bleu_score_epoch)
# print("rouge_score_epoch", rouge_score_epoch)

In [ ]:
rouge1_precision_epoch=[round(rouge_output["rouge1"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rouge1_recall_epoch=[round(rouge_output["rouge1"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rouge1_fmeasure_epoch=[round(rouge_output["rouge1"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rouge2_precision_epoch=[round(rouge_output["rouge2"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rouge2_recall_epoch=[round(rouge_output["rouge2"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rouge2_fmeasure_epoch=[round(rouge_output["rouge2"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rougeL_precision_epoch=[round(rouge_output["rougeL"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rougeL_recall_epoch=[round(rouge_output["rougeL"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rougeL_fmeasure_epoch=[round(rouge_output["rougeL"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

rougeLsum_precision_epoch=[round(rouge_output["rougeLsum"].mid.precision, 4) for rouge_output in rouge_score_epoch]
rougeLsum_recall_epoch=[round(rouge_output["rougeLsum"].mid.recall, 4) for rouge_output in rouge_score_epoch]
rougeLsum_fmeasure_epoch=[round(rouge_output["rougeLsum"].mid.fmeasure, 4) for rouge_output in rouge_score_epoch]

            
bleu_avg_score_epoch= [round(bleu_output["bleu"], 4) for bleu_output in bleu_score_epoch]
bleu_1_score_epoch=[round(bleu_output["precisions"][0], 4) for bleu_output in bleu_score_epoch]
bleu_2_score_epoch=[round(bleu_output["precisions"][1], 4) for bleu_output in bleu_score_epoch]
bleu_3_score_epoch=[round(bleu_output["precisions"][2], 4) for bleu_output in bleu_score_epoch]
bleu_4_score_epoch=[round(bleu_output["precisions"][3], 4) for bleu_output in bleu_score_epoch]

meteor_score_epoch=[round(meteor_avg, 4) for meteor_avg in meteor_score_epoch]

In [ ]:
import pandas as pd
df = pd.DataFrame({
    'rouge1_precision_epoch': rouge1_precision_epoch,
    'rouge1_recall_epoch': rouge1_recall_epoch,
    'rouge1_fmeasure_epoch': rouge1_fmeasure_epoch,

    'rouge2_precision_epoch': rouge2_precision_epoch,
    'rouge2_recall_epoch': rouge2_recall_epoch,
    'rouge2_fmeasure_epoch': rouge2_fmeasure_epoch,

    'rougeL_precision_epoch': rougeL_precision_epoch,
    'rougeL_recall_epoch': rougeL_recall_epoch,
    'rougeL_fmeasure_epoch': rougeL_fmeasure_epoch,

    'rougeLsum_precision_epoch': rougeLsum_precision_epoch,
    'rougeLsum_recall_epoch': rougeLsum_recall_epoch,
    'rougeLsum_fmeasure_epoch': rougeLsum_fmeasure_epoch,

    'bleu_avg_score_epoch': bleu_avg_score_epoch,
    'bleu_1_score_epoch': bleu_1_score_epoch,
    'bleu_2_score_epoch': bleu_2_score_epoch,
    'bleu_3_score_epoch': bleu_3_score_epoch,
    'bleu_4_score_epoch': bleu_4_score_epoch,
    
    'meteor_score_epoch': meteor_score_epoch,
})

df.to_csv('metrics_epoch_soft_prompt_3407.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

plt.plot(test_losses_epoch, '*-r')

plt.plot(rouge1_precision_epoch, '*--g')
plt.plot(rouge1_recall_epoch, 'x--g')
plt.plot(rouge1_fmeasure_epoch, 'o--g')

plt.plot(bleu_avg_score_epoch, '*:b')
plt.plot(bleu_1_score_epoch, 'x:b')
plt.plot(bleu_2_score_epoch, '+:b')
plt.plot(bleu_3_score_epoch, 'o:b')
plt.plot(bleu_4_score_epoch, 'd:b')

plt.plot(meteor_score_epoch, '*-.y')

plt.show()

In [ ]:
inputs = tokenizer(test_sensor_data[0], return_tensors="pt").to(device)
print(test_sensor_data[0])

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1).to(device), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1).to(device), inputs['attention_mask']], 1)
inputs['decoder_input_ids']=torch.full((1, 100), 1).to(device)
 
outputs = model(**inputs)
print(outputs.logits.shape)
output_ids=outputs.logits[:, n_tokens:, :]
print(output_ids.shape)
max_ids=output_ids.argmax(-1).detach().cpu().numpy().tolist()
# max_ids[0]=[ids for ids in max_ids[0] if ids!=0]
print(tokenizer.batch_decode(max_ids))

In [ ]:
from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d%m_%H%M%S")
torch.save(model.state_dict(), "./output/T5_"+dt_string)

In [ ]:
model = MyModel(num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-2)
output_model = './models/model_xlnet_mid.pth'

# save
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(model, optimizer)

# load
checkpoint = torch.load(output_model, map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Evaluation

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model.load_state_dict(torch.load("./output/T5_2404_092246"))
tokenizer = T5Tokenizer.from_pretrained("t5-base")
n_tokens = len(WARNING_DL)
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.to(device)

In [ ]:
from datasets import load_metric

In [ ]:
all_true_labels = []
all_pred_labels = []
output_max_pred=[]
test_losses = []
with torch.no_grad():
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, test_sensor_data, test_templates)), total=dev_total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        
        output_ids=outputs.logits[:, n_tokens:, :]
        max_pred_ids=outputs.logits[:, n_tokens:, :].argmax(-1).detach().cpu().numpy().tolist()
        max_pred_ids[0]=[ids for ids in max_pred_ids[0] if ids!=0]
        output_max_pred.extend(tokenizer.batch_decode(max_pred_ids))
        
        loss = outputs.loss
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        test_losses.append(loss_value)
        pred_labels = outputs['logits'][:, :, :].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels
        pbar.set_description(f'dev: loss={np.mean(test_losses):.4f}')

In [ ]:
print(output_max_pred)

In [ ]:
bleu=load_metric("bleu")
rouge=load_metric("rouge")

In [ ]:
bleu_preds=[tokenizer.tokenize(pred) for pred in output_max_pred]
bleu_refers=[[tokenizer.tokenize(label)] for label in test_templates]


In [ ]:
bleu_score=bleu.compute(predictions=bleu_preds, references=bleu_refers)
print(bleu_score)
rouge_score=rouge.compute(predictions=output_max_pred, references=test_templates)
print(rouge_score)

In [ ]:
tokenizer.encode("Hello world")